In [ ]:
# make missense codon tables

# this uses a corrected function to make missense variants

from itertools import product
from Bio.Seq import Seq
import pandas as pd

iupac_dict = {'A':'A','C':'C','G':'G','T':'T','AC':'M','AG':'R','AT':'W','CG':'S','CT':'Y','GT':'K','ACG':'V','ACT':'H','AGT':'D','CGT':'B','ACGT':'N'}


# AA LEVEL (64 entries)
nucleotides = 'ACGT'

codon_list = []
aa_list = []
iupac_codon_list = []
missense_aa_list = []


for codon in list(''.join(w) for w in product(nucleotides, repeat=3)):
    aa = Seq(codon).translate(table='Standard')[0]
    new_aas = []
    iupac_codon = []
    
    # loop through each position in codon
    for position in range(3):
        iupac_n = []
        
        # looping through each nucleotide
        for n in nucleotides:
            new_codon = codon[:position] + n + codon[position + 1:]
            new_aa = Seq(new_codon).translate(table='Standard')[0]
            if new_aa != aa:
                new_aas.append(new_aa)
                iupac_n.append(n)
            else:
                continue
        
        # iupac symbols
        if not iupac_n: # check if the list is empty
            iupac_codon.append(codon[position])
        else:
            for i in iupac_dict.keys():
                if set(i) == set(iupac_n):
                    iupac_codon.append(iupac_dict[i])
    
    # make assignments here
    codon_list.append(codon)
    aa_list.append(aa)
    iupac_codon_list.append(''.join(iupac_codon))
    missense_aa_list.append(''.join(set(new_aas)))

# create AA Level dictionary
aa_level_dict = {
    'codon':codon_list, 
    'aa':aa_list, 
    'iupac_codon':iupac_codon_list,
    'missense_aa':missense_aa_list   
}

df = pd.DataFrame.from_dict(aa_level_dict)
df.to_csv('data/aa_missense_table.csv', index=False)

# CODON LEVEL
# df: codon, aa, position, nucleotides, missense_codons, missense_aa, iupac, iupac_codon
nucleotides = 'ACGT'

codon_list = []
aa_list = []
position_list = []
nucleotides_list = []
missense_codons_list = []
missense_aa_list = []
iupac_list = []
iupac_codon_list = []

for codon in list(''.join(w) for w in product(nucleotides, repeat=3)):
    aa = Seq(codon).translate(table='Standard')[0]
    
    # loop through each position in codon
    for position in range(3):      
        new_aas = []
        iupac_n = []
        new_codons = []
        # looping through each nucleotide
        for i in nucleotides:
            new_codon = codon[:position] + i + codon[position + 1:]
            new_aa = Seq(new_codon).translate(table='Standard')[0]
            if not new_aa == aa:
                new_aas.append(new_aa)
                iupac_n.append(i)
                new_codons.append(new_codon)
            else:
                continue
        
        #iupac
        if not iupac_n: # check if iupac_n is empty
            iupac = codon[position]
        else:
            for i in iupac_dict.keys():
                if set(i) == set(iupac_n):
                    iupac = iupac_dict[i]
      
        # make assignments here
        codon_list.append(codon)
        aa_list.append(aa)
        position_list.append(position)
        nucleotides_list.append(''.join(iupac_n))
        missense_codons_list.append(' '.join(new_codons))
        missense_aa_list.append(''.join(set(new_aas)))
        
        
        for i in iupac_dict.keys():
            if set(i) == set(iupac_n):
                iupac = iupac_dict[i]
        iupac_list.append(iupac)
        iupac_codon_list.append(codon[:position] + iupac + codon[position + 1:])

# create dictionary
codon_level_dict = {
    'codon':codon_list, 
    'aa':aa_list, 
    'position':position_list,
    'missense_nucleotides':nucleotides_list,
    'missense_codons':missense_codons_list,
    'missense_aa':missense_aa_list,
    'iupac':iupac_list,
    'iupac_codon':iupac_codon_list
}

df = pd.DataFrame.from_dict(codon_level_dict)
df.to_csv('data/codon_missense_table.csv', index=False)